# Statistical testing in binary classification

## Neccessary stuff
1. Train two or more models.
1. Evaluate trained models on one or more test datasets (it's a little bit tricky when you want to make several datasets from one).
1. Ask a question you want to answer about this models.
1. Choose statistical test equivalent to your question.
1. Measure neccessary things for this test.
1. Assume some significance level, e.g. $\alpha = 0.05$. 
1. Calculate p-value for the test.

In [1]:
%load_ext dotenv
%dotenv

In [3]:
import os
import shutil
import numpy as np
import tensorflow as tf
import scipy
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from statsmodels.stats.contingency_tables import mcnemar
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Two models, one data set, what can I do?

### Data

In [4]:
# The path to store trained models
models_dir = './models/'
if not os.path.exists(models_dir):
    os.mkdir(models_dir)

# The path to the directory where the original dataset was uncompressed
original_dataset_dir = './Dogs-vs-Cats-1'
original_cat_dir = './Dogs-vs-Cats-1/Cat'
original_dog_dir = './Dogs-vs-Cats-1/Dog'

# The directory where we will store our smaller dataset
base_dir = './Dogs-vs-Cats-1/working'
if not os.path.exists(base_dir):
    os.mkdir(base_dir)

# Directories for our training, validation and test splits
train_dir = os.path.join(base_dir, 'train')
if not os.path.exists(train_dir):
    os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
if not os.path.exists(test_dir):
    os.mkdir(test_dir)

# Directory with training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')
if not os.path.exists(train_cats_dir):
    os.mkdir(train_cats_dir)

# Directory with training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')
if not os.path.exists(train_dogs_dir):
    os.mkdir(train_dogs_dir)

# Directory with validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')
if not os.path.exists(validation_cats_dir):
    os.mkdir(validation_cats_dir)
    
# Directory with validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
if not os.path.exists(validation_dogs_dir):
    os.mkdir(validation_dogs_dir)

# Directory with test cat pictures
test_cats_dir = os.path.join(test_dir, 'cats')
if not os.path.exists(test_cats_dir):
    os.mkdir(test_cats_dir)

# Directory with test dog pictures
test_dogs_dir = os.path.join(test_dir, 'dogs')
if not os.path.exists(test_dogs_dir):
    os.mkdir(test_dogs_dir)

In [16]:
# Copy first 1000 cat images to train_cats_dir
fnames = ['{}.jpg'.format(i) for i in range(1001)]
for fname in fnames:
    if fname == '666.jpg':
        continue
    src = os.path.join(original_cat_dir, fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)

# Copy next 500 cat images to validation_cats_dir
fnames = ['{}.jpg'.format(i) for i in range(1001, 1501)]
for fname in fnames:
    src = os.path.join(original_cat_dir, fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 cat images to test_cats_dir
fnames = ['{}.jpg'.format(i) for i in range(1501, 2001)]
for fname in fnames:
    src = os.path.join(original_cat_dir, fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy first 1000 dog images to train_dogs_dir
fnames = ['{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(original_dog_dir, fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to validation_dogs_dir
fnames = ['{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(original_dog_dir, fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
# Copy next 500 dog images to test_dogs_dir
fnames = ['{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(original_dog_dir, fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)

In [5]:
# More test sets from first dataset
test_dirs = []
for i in range(19):
    test_dirs.append(os.path.join(base_dir, f'test_{i}'))
    if not os.path.exists(test_dirs[i]):
        os.mkdir(test_dirs[i])

    temp_test_cats_dir = os.path.join(test_dirs[i], 'cats')
    if not os.path.exists(temp_test_cats_dir):
        os.mkdir(temp_test_cats_dir)

    temp_test_dogs_dir = os.path.join(test_dirs[i], 'dogs')
    if not os.path.exists(temp_test_dogs_dir):
        os.mkdir(temp_test_dogs_dir)

In [49]:
for i in range(len(test_dirs)):
    temp_test_cats_dir = os.path.join(test_dirs[i], 'cats')
    fnames = ['{}.jpg'.format(i) for i in range(2001+i*500, 2501+i*500)]
    for fname in fnames:
        src = os.path.join(original_cat_dir, fname)
        dst = os.path.join(temp_test_cats_dir, fname)
        shutil.copyfile(src, dst)

    temp_test_dogs_dir = os.path.join(test_dirs[i], 'dogs')
    fnames = ['{}.jpg'.format(i) for i in range(2000+i*500, 2500+i*500)]
    for fname in fnames:
        src = os.path.join(original_dog_dir, fname)
        dst = os.path.join(temp_test_dogs_dir, fname)
        shutil.copyfile(src, dst)

In [6]:
img_rows = 150
img_cols = 150

# data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)
test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size, 
        class_mode='binary')

val_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size, 
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        shuffle=False,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [7]:
img_rows = 150
img_cols = 150

# Additional test generators
test_generators = []
for dir in test_dirs:
    test_generators.append(test_datagen.flow_from_directory(
        dir,
        target_size=(img_rows, img_cols),
        batch_size=batch_size,
        shuffle=False,
        class_mode='binary'))

Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


### Models

In [6]:
# First model
model1 = Sequential()
model1.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_rows, img_cols, 3)))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(64, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(128, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(256, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Conv2D(256, (3, 3), activation='relu'))
model1.add(MaxPooling2D((2, 2)))
model1.add(Flatten())
model1.add(Dropout(0.5))
model1.add(Dense(512, activation='relu'))
model1.add(Dense(1, activation='sigmoid'))

In [7]:
model1.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=3e-4),
              metrics=['acc'])

model1.fit(
      train_generator,
      batch_size=batch_size,
      epochs=30,
      validation_data=val_generator)

Epoch 1/30


/home/tosiek/Documents/VI_semestr/GiGSN/statistical-significance-ml/.venv/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1718034791.795290   13587 service.cc:145] XLA service 0x72a8cc00ac70 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1718034791.795315   13587 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce GTX 1650 Ti, Compute Capability 7.5
2024-06-10 17:53:11.856243: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-06-10 17:53:12.094004: I external/local_xla/xla/stream_executor/cuda/cud

 1/63 ━━━━━━━━━━━━━━━━━━━━ 9:27 9s/step - acc: 0.6562 - loss: 0.6909

I0000 00:00:1718034798.751011   13587 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


63/63 ━━━━━━━━━━━━━━━━━━━━ 25s 250ms/step - acc: 0.5094 - loss: 0.6946 - val_acc: 0.5030 - val_loss: 0.6922
Epoch 2/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 133ms/step - acc: 0.5372 - loss: 0.6908 - val_acc: 0.5030 - val_loss: 0.6884
Epoch 3/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 135ms/step - acc: 0.5322 - loss: 0.6904 - val_acc: 0.5700 - val_loss: 0.6800
Epoch 4/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 136ms/step - acc: 0.5822 - loss: 0.6827 - val_acc: 0.6310 - val_loss: 0.6705
Epoch 5/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 135ms/step - acc: 0.5868 - loss: 0.6760 - val_acc: 0.6060 - val_loss: 0.6573
Epoch 6/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 146ms/step - acc: 0.5991 - loss: 0.6635 - val_acc: 0.6420 - val_loss: 0.6270
Epoch 7/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 135ms/step - acc: 0.6068 - loss: 0.6486 - val_acc: 0.5040 - val_loss: 0.8120
Epoch 8/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 139ms/step - acc: 0.6177 - loss: 0.6434 - val_acc: 0.6600 - val_loss: 0.6114
Epoch 9/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 138ms/step - acc: 0.

In [8]:
model1.save(models_dir+"binary_model1.h5")

In [9]:
# Second model
model2 = Sequential()
model2.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_rows, img_cols, 3)))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(64, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(128, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(256, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Conv2D(256, (3, 3), activation='relu'))
model2.add(MaxPooling2D((2, 2)))
model2.add(Flatten())
model2.add(Dropout(0.5))
model2.add(Dense(512, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

In [10]:
model2.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=3e-4),
              metrics=['acc'])

model2.fit(
      train_generator,
      batch_size=batch_size,
      epochs=30,
      validation_data=val_generator)

Epoch 1/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 17s 194ms/step - acc: 0.5131 - loss: 0.6934 - val_acc: 0.5090 - val_loss: 0.6891
Epoch 2/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 147ms/step - acc: 0.5520 - loss: 0.6891 - val_acc: 0.5830 - val_loss: 0.6750
Epoch 3/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 139ms/step - acc: 0.5871 - loss: 0.6749 - val_acc: 0.6200 - val_loss: 0.6494
Epoch 4/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 140ms/step - acc: 0.6323 - loss: 0.6492 - val_acc: 0.6300 - val_loss: 0.6321
Epoch 5/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 142ms/step - acc: 0.6410 - loss: 0.6289 - val_acc: 0.6580 - val_loss: 0.6235
Epoch 6/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 140ms/step - acc: 0.6644 - loss: 0.6230 - val_acc: 0.6760 - val_loss: 0.6075
Epoch 7/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 145ms/step - acc: 0.6680 - loss: 0.6172 - val_acc: 0.6550 - val_loss: 0.6143
Epoch 8/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 143ms/step - acc: 0.6753 - loss: 0.6252 - val_acc: 0.6600 - val_loss: 0.5965
Epoch 9/30
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 142ms/

In [11]:
model2.save(models_dir+"binary_model2.h5")

### Want to know if one model is better than other?
Perform McNemar's test! It's simple statistic based on *False Negatives* clasified only one of the two models. The statistic of this test is calucated by equation below:
$$\chi^2 = \frac{(|b-c|-1)^2}{b+c}$$
* b - *False Negatives* clasified by first model, but not by the second,
* c - *False Negatives* clasified by second model, but not the first.

This test follows chi-squared distribution with one degree of freedom when $b+c\geq20$, and binomial distribution in the other case.  

In [6]:
# Load models
model1 = load_model(models_dir+"binary_model1.h5")
model2 = load_model(models_dir+"binary_model2.h5")

In [28]:
# Predict 
y_true = test_generator.labels
size = len(y_true)
y_pred1 = np.array([model1.predict(test_generator) > 0.5], dtype=int).reshape((size,))
y_pred2 = np.array([model2.predict(test_generator) > 0.5], dtype=int).reshape((size,))

32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step


In [29]:
# Calculate b and c
b = 0
c = 0
for i in range(size):
    if y_true[i] == 1:
        if y_pred1[i] == 0 and y_pred2[i] == 1:
            b += 1
        elif y_pred1[i] == 1 and y_pred2[i] == 0:
            c += 1 

print("False negatives classified by first model, but not by the second:", b)
print("False negatives classified by second model, but not by the first:", c)

False negatives classified by first model, but not by the second: 66
False negatives classified by second model, but not by the first: 19


In [30]:
# Calulate test statistic
chi2 = (abs(b-c) - 1)**2/(b+c)
p = 2 * min(scipy.stats.chi2.sf(chi2, 1), scipy.stats.chi2.cdf(chi2, 1))
print(p)

1.2113390294598264e-06


### Difference between ROC curves
There is also a posibility to check if there is a significant difference between ROC curves, perfolming DeLong's test. This test is a little bit more demanding computionally, because we need to estimate some values. Test statistics is obtained by equation below.
$$z_D=\frac{\hat\theta_1-\hat\theta_2}{\sqrt{Var(\hat\theta_1)+Var(\hat\theta_2)-2Cov(\hat\theta_1,\hat\theta_2)}}$$

In the equation above, $\hat\theta_k$ is an estimated *Area under ROC Curve* of k-th model. We can obtain this value by formula given below.
$$\hat\theta=\frac{1}{mn}\sum_{i=1}^m\sum_{j=1}^n\Psi(Y_{i1},Y_{j0}), \Psi(Y_{i1},Y_{j0})=\begin{cases}
      1 & Y_{i1} > Y_{j0}\\
      \frac{1}{2} & Y_{i1} = Y_{j0}\\
      0 & Y_{i1} < Y_{j0}
    \end{cases}$$ 
Where $Y_{i1}$ is numeric prediction of i-th truly positive instance, and $Y_{j0}$ is numeric prediction of j-th truly negative instance, and $m$ and $n$ are, correspondingly, numbers of this instances. 

Calculating estimates of variance and coveriance is a little bit more tricky. First we need to caluclate *structual components*. Structural components are defined separetly for truly positive and negative instances. Structural element of r-th model for i-th truly positive instance is calculated by the formula given below. 
$$V^r_{10}(Y_{i1})=\frac{1}{n}\sum_{j=1}^n\Psi(Y_{i1}, Y_{j0})$$ 

Similarly we define structural element for j-th truly negative instance.
$$V^r_{01}(Y_{j0})=\frac{1}{m}\sum_{i=1}^m\Psi(Y_{i1}, Y_{j0})$$

Having structural elements defined we can compute matrices $S_{10}$ and $S_{01}$. Value in r-th row and s-th column is given by the following equations.
$$S^{(r,s)}_{10}=\frac{1}{m-1}\sum^m_{i=1}[V^r_{10}(Y_{i1})-\hat\theta_r][V^s_{10}(Y_{i1})-\hat\theta_s]$$
$$S^{(r,s)}_{01}=\frac{1}{n-1}\sum^n_{j=1}[V^r_{01}(Y_{j0})-\hat\theta_r][V^s_{01}(Y_{j0})-\hat\theta_s]$$

Once we calculate matricies $S_{10}$ and $S_{01}$, we can calculate matrix S from which we read values for estimated values of variance and covariance. Formula for this matrix is presented below.
$$S=\frac{1}{m}S_{10}+\frac{1}{n}S_{01}=\begin{Bmatrix}
Var(\hat\theta_1) & Cov(\hat\theta_1,\hat\theta_2) \\
Cov(\hat\theta_2,\hat\theta_1) & Var(\hat\theta_2)
\end{Bmatrix}$$

In [25]:
def psi(y1, y0):
    if y1 > y0:
        return 1
    if y1 == y0:
        return 0.5
    return 0

def v10(y1, y0):
    n = len(y0)
    sum = 0
    for j in y0:
        sum += psi(y1, j)
    return sum / n

def v01(y0, y1):
    m = len(y1)
    sum = 0
    for i in y1:
        sum += psi(i, y0)
    return sum / m

def s10(y1, y0, thetas):
    m = len(y1[0])
    s_ = np.zeros((2,2))
    for r in range(2):
        for s in range(2):
            for i in range(m):
               s_[r, s] += (v10(y1[r][i], y0[r])-thetas[r])*(v10(y1[s][i], y0[s])-thetas[s])
            s_[r, s] /= (m-1)
    return s_

def s01(y1, y0, thetas):
    n = len(y0[0])
    s_ = np.zeros((2,2))
    for r in range(2):
        for s in range(2):
            for j in range(n):
               s_[r, s] += (v01(y0[r][j], y1[r])-thetas[r])*(v01(y0[s][j], y1[s])-thetas[s])
            s_[r, s] /= (n-1)
    return s_

def theta(y1, y0):
    m = len(y1)
    n = len(y0)
    sum = 0
    for i in range(m):
        for j in range(n):
            sum += psi(y1[i], y0[j])
    return sum/(m*n)

def delong_test(y_true, y_pred1, y_pred2):
    y1 = [[], []]
    y0 = [[], []]
    for i in range(len(y_true)):
        if y_true[i] == 0:
            y0[0].append(y_pred1[i])
            y0[1].append(y_pred2[i])
        else:
            y1[0].append(y_pred1[i])
            y1[1].append(y_pred2[i])
    thetas = [theta(y1[0], y0[0]), theta(y1[1], y0[1])]
    m = len(y1[0])
    n = len(y0[0])
    s = s10(y1, y0, thetas)/m + s01(y1, y0, thetas)/n
    var1 = s[0, 0]
    var2 = s[1, 1]
    cov = s[0, 1]
    z = (thetas[0]-thetas[1])/(var1 + var2 - 2*cov)**0.5
    p = 2*min(scipy.stats.norm.sf(z), scipy.stats.norm.cdf(z))
    return p

In [10]:
# Load models
model1 = load_model(models_dir+"binary_model1.h5")
model2 = load_model(models_dir+"binary_model2.h5")

In [12]:
# Predict 
y_true = test_generator.labels
y_pred1 = model1.predict(test_generator)
y_pred2 = model2.predict(test_generator)

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step
32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step


In [26]:
delong_test(y_true, y_pred1, y_pred2)

2.8305778345316294e-05

## What if we have two models and more data?
At this point we have two possible questions we can answer. First:

### Is there a significant difference in this models.
If we want to answer this question, we can use Wilcoxon test!
First we need to evaluate choosen metric for all test sets. After that we calculate differences between metrics, for corresponding test sets. After that, we order this differences by absoulte value, and then calculate two values, using formula below, knowing that $d_i$ is difference between metrics obtained on i-th set, and $rank(|d_i|)$ is place of that difference in our series. 
$$R^+=\sum_{d_i>0}rank(|d_i|)$$
$$R^-=\sum_{d_i<0}rank(|d_i|)$$

As test statistic we can take value: 
$$T=min\{R^+, R^-\}$$
Or, if we have large number (n) of test cases, we can use statistic $z$, which follows normal distribution:
$$z=\frac{T-\frac{n(n+1)}{4}}{\sqrt(\frac{n(n+1)(2n+1)}{24})}$$ 

To compute this statistics, we don't need to do everything manually this time, we can use method from scipy.stats: wicloxon. By default this method uses statistic $T$, when number of test cases is less or equal to 50, but we can choose if we want to use statistic $T$ or $z$, by passing argument *method* with value - respectively - *exact* and *approx*.

In [42]:
# Load models
model1 = load_model(models_dir+"binary_model1.h5")
model2 = load_model(models_dir+"binary_model2.h5")

model1.compile(optimizer=model1.optimizer,
                        loss=model1.loss,
                        metrics=['acc', 'auc', 'precision', 'recall'])
model2.compile(optimizer=model2.optimizer,
                        loss=model2.loss,
                        metrics=['acc', 'auc', 'precision', 'recall'])

In [43]:
model1_metrics = []
model2_metrics = []
for gen in test_generators:
    model1_metrics.append(model1.evaluate(gen, verbose=0)[1:])
    model2_metrics.append(model2.evaluate(gen, verbose=0)[1:])

print(f'First model accuracies: {model1_metrics}')
print(f'Second model accuracies: {model2_metrics}')

/home/tosiek/Documents/VI_semestr/GiGSN/statistical-significance-ml/.venv/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:890: UserWarning: Truncated File Read
  warnings.warn(str(msg))


First model accuracies: [[0.7699999809265137, 0.847681999206543, 0.8139534592628479, 0.699999988079071], [0.746999979019165, 0.8079699277877808, 0.7905882596969604, 0.671999990940094], [0.7400000095367432, 0.8189380168914795, 0.7898550629615784, 0.6539999842643738], [0.7490000128746033, 0.8129520416259766, 0.792941153049469, 0.6740000247955322], [0.7540000081062317, 0.8391139507293701, 0.8038277626037598, 0.671999990940094], [0.75, 0.8264439702033997, 0.7765486836433411, 0.7020000219345093], [0.7289999723434448, 0.8130620121955872, 0.7584649920463562, 0.671999990940094], [0.7390000224113464, 0.8294360637664795, 0.7697516679763794, 0.6819999814033508], [0.7630000114440918, 0.8403159976005554, 0.8036951422691345, 0.6959999799728394], [0.7540000081062317, 0.8262599110603333, 0.7834821343421936, 0.7020000219345093], [0.777999997138977, 0.8302860856056213, 0.8130630850791931, 0.722000002861023], [0.7139999866485596, 0.8054440021514893, 0.7661691308021545, 0.6159999966621399], [0.74699997901

In [39]:
model1_metrics = np.array(model1_metrics)
model2_metrics = np.array(model2_metrics)

model1_acc = model1_metrics[:, 0]
model2_acc = model2_metrics[:, 0]

model1_auc = model1_metrics[:, 1]
model2_auc = model2_metrics[:, 1]

model1_prec = model1_metrics[:, 2]
model2_prec = model2_metrics[:, 2]

model1_rec = model1_metrics[:, 3]
model2_rec = model2_metrics[:, 3]

In [40]:
stats, p = scipy.stats.wilcoxon(model1_acc, model2_acc)
print(f"Test statistic: {stats}")
print(f"Test pvalue: {p}")

Test statistic: 0.0
Test pvalue: 3.814697265625e-06


In [41]:
stats, p = scipy.stats.wilcoxon(model1_auc, model2_auc)
print(f"Test statistic: {stats}")
print(f"Test pvalue: {p}")

Test statistic: 0.0
Test pvalue: 3.814697265625e-06


In [42]:
stats, p = scipy.stats.wilcoxon(model1_prec, model2_prec)
print(f"Test statistic: {stats}")
print(f"Test pvalue: {p}")

Test statistic: 69.0
Test pvalue: 0.312408447265625


In [43]:
stats, p = scipy.stats.wilcoxon(model1_rec, model2_rec)
print(f"Test statistic: {stats}")
print(f"Test pvalue: {p}")

Test statistic: 0.0
Test pvalue: 3.814697265625e-06


There is also a second question we can ask.

### Is there a significant difference between variances of two models?
To answer this question we can use f-test. But there is one restriction: data cannot have normal distribution. To check, if data is normal we can perform *Shapiro-Wilk* test. Shapriro-Wilk test has a few steps. First we need to sort our test values ($x$) ascending. Then we have to compute: $SSE=\sum_{i=1}^n(x_i-\hat x)$. Then we need to calulate value $m$, based of the number of test cases ($n$): $m=\frac{n}{2}$ if $n$ is even, $m=\frac{n-1}{2}$ in the other case. After that we compute $b=\sum_{i=1}^ma_i(x_{n+1-i}-x_i)$, $a_i$ we have to read from special table. Having value $b$ we can finally compute test statistics: $W=\frac{b^2}{SSE}$. Fortunately we can perform this test using method *shapiro* from *scipy.stats*. 

In [44]:
# Load models
model1 = load_model(models_dir+"binary_model1.h5")
model2 = load_model(models_dir+"binary_model2.h5")

model1.compile(optimizer=model1.optimizer,
                        loss=model1.loss,
                        metrics=['acc', 'auc', 'precision', 'recall'])
model2.compile(optimizer=model2.optimizer,
                        loss=model2.loss,
                        metrics=['acc', 'auc', 'precision', 'recall'])

In [45]:
model1_metrics = []
model2_metrics = []
for gen in test_generators:
    model1_metrics.append(model1.evaluate(gen, verbose=0)[1:])
    model2_metrics.append(model2.evaluate(gen, verbose=0)[1:])

print(f'First model accuracies: {model1_metrics}')
print(f'Second model accuracies: {model2_metrics}')

/home/tosiek/Documents/VI_semestr/GiGSN/statistical-significance-ml/.venv/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:890: UserWarning: Truncated File Read
  warnings.warn(str(msg))


First model accuracies: [[0.7699999809265137, 0.847681999206543, 0.8139534592628479, 0.699999988079071], [0.746999979019165, 0.8079699277877808, 0.7905882596969604, 0.671999990940094], [0.7400000095367432, 0.8189380168914795, 0.7898550629615784, 0.6539999842643738], [0.7490000128746033, 0.8129520416259766, 0.792941153049469, 0.6740000247955322], [0.7540000081062317, 0.8391139507293701, 0.8038277626037598, 0.671999990940094], [0.75, 0.8264439702033997, 0.7765486836433411, 0.7020000219345093], [0.7289999723434448, 0.8130620121955872, 0.7584649920463562, 0.671999990940094], [0.7390000224113464, 0.8294360637664795, 0.7697516679763794, 0.6819999814033508], [0.7630000114440918, 0.8403159976005554, 0.8036951422691345, 0.6959999799728394], [0.7540000081062317, 0.8262599110603333, 0.7834821343421936, 0.7020000219345093], [0.777999997138977, 0.8302860856056213, 0.8130630850791931, 0.722000002861023], [0.7139999866485596, 0.8054440021514893, 0.7661691308021545, 0.6159999966621399], [0.74699997901

In [46]:
model1_metrics = np.array(model1_metrics)
model2_metrics = np.array(model2_metrics)

model1_acc = model1_metrics[:, 0]
model2_acc = model2_metrics[:, 0]

model1_auc = model1_metrics[:, 1]
model2_auc = model2_metrics[:, 1]

model1_prec = model1_metrics[:, 2]
model2_prec = model2_metrics[:, 2]

model1_rec = model1_metrics[:, 3]
model2_rec = model2_metrics[:, 3]

In [47]:
def check_normality(y, metric, model):
    _, p = scipy.stats.shapiro(y)
    if p < 0.001:
        print(f'{metric} of {model} model are normally distrtibuted.')
    else:
        print(f'{metric} of {model} model are not normally distrtibuted.')

# checking if accuracies are normally distributed
check_normality(model1_acc, "Accuracies", "first")
check_normality(model2_acc, "Accuracies", "second")
print()
# checking if auces are normally distributed
check_normality(model1_auc, "AUCes", "first")
check_normality(model2_auc, "AUCes", "second")
print()
# checking if precisions are normally distributed
check_normality(model1_prec, "Precisions", "first")
check_normality(model2_prec, "Precisions", "second")
print()
# checking if recalls are normally distributed
check_normality(model1_rec, "Recalls", "first")
check_normality(model2_rec, "Recalls", "second")
print()

Accuracies of first model are not normally distrtibuted.
Accuracies of second model are not normally distrtibuted.

AUCes of first model are not normally distrtibuted.
AUCes of second model are not normally distrtibuted.

Precisions of first model are not normally distrtibuted.
Precisions of second model are not normally distrtibuted.

Recalls of first model are not normally distrtibuted.
Recalls of second model are not normally distrtibuted.



In our case all of the metrics results are not normally distributed. Now, we can perform F-test. To calculate test statistic we use the equation below.
$$F=\frac{\sigma_1^2}{\sigma_2^2}$$
This value follows f-distribution with $n-1$ and $n-1$ degrees of freedom, where $n$ is the number of test samples.

In [53]:
var1 = np.var(model1_acc)
var2 = np.var(model2_acc)
F = var1/var2
n = len(model1_acc)
p = 2*min(scipy.stats.f.sf(F, n-1, n-1), scipy.stats.f.cdf(F, n-1, n-1))
p

0.20397118240415707

In [55]:
var1 = np.var(model1_auc)
var2 = np.var(model2_auc)
F = var1/var2
n = len(model1_auc)
p = 2*min(scipy.stats.f.sf(F, n-1, n-1), scipy.stats.f.cdf(F, n-1, n-1))
p

0.4545874378951895

In [56]:
var1 = np.var(model1_prec)
var2 = np.var(model2_prec)
F = var1/var2
n = len(model1_prec)
p = 2*min(scipy.stats.f.sf(F, n-1, n-1), scipy.stats.f.cdf(F, n-1, n-1))
p

0.3245173052057107

In [57]:
var1 = np.var(model1_rec)
var2 = np.var(model2_rec)
F = var1/var2
n = len(model1_rec)
p = 2*min(scipy.stats.f.sf(F, n-1, n-1), scipy.stats.f.cdf(F, n-1, n-1))
p

0.23245616713399564